In [18]:
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
import nltk
import string
import multiprocessing
import time
cores = multiprocessing.cpu_count()

In [19]:
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
from sklearn import metrics 
from sklearn.metrics import classification_report
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import gensim.models.doc2vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [20]:
assert gensim.models.doc2vec.FAST_VERSION > -1

In [21]:
def remove_punc(post):
    punc_num = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_num])

In [22]:
def remove_stop_words(tokens):
    sss = set(stopwords.words('english'))
    return [t for t in tokens if t not in sss]

In [23]:
mbti = pd.read_csv('train.csv')

In [24]:
all_mbti = []
for i,r in mbti.iterrows():
    for comment in r['posts'].split('|||'):
        all_mbti.append([r['type'],comment])
all_mbti = pd.DataFrame(all_mbti, columns=['type', 'post'])

In [25]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'

In [26]:
all_mbti['post'] = all_mbti['post'].replace(to_replace = pattern_url, value = subs_url, regex = True)
all_mbti['post'] = all_mbti['post'].str.lower()
all_mbti['post'] = all_mbti['post'].apply(remove_punc)

In [27]:
tokeniser = TreebankWordTokenizer()
all_mbti['tokens'] = all_mbti['post'].apply(tokeniser.tokenize)

In [29]:
all_mbti = all_mbti[all_mbti['post']!='urlweb']
all_mbti.head()

,type,post,tokens
2,INFJ,enfp and intj moments urlweb sportscenter no...,"[enfp, and, intj, moments, urlweb, sportscente..."
3,INFJ,what has been the most lifechanging experience...,"[what, has, been, the, most, lifechanging, exp..."
4,INFJ,urlweb urlweb on repeat for most of today,"[urlweb, urlweb, on, repeat, for, most, of, to..."
5,INFJ,may the perc experience immerse you,"[may, the, perc, experience, immerse, you]"
6,INFJ,the last thing my infj friend posted on his fa...,"[the, last, thing, my, infj, friend, posted, o..."


In [30]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(all_mbti['tokens'])]

In [31]:
max_epochs = 50
vec_size = 100
alpha = 0.043

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                window = 4 ,
                min_alpha=0.033,
                min_count=3,
                dm =1,
                max_vocab_size=20000,
                workers=cores,
                negative = 5)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
len(model.wv.index2word)

In [ ]:
model.iter

In [37]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
        # decrease the learning rate
    model.alpha -= 0.0002
    print('Model Alpha: ',model.alpha)
                # fix the learning rate, no decay
    model.min_alpha = model.alpha
#model.save("no_url.model_w4")
print("Model Saved")

iteration 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


Model Alpha:  0.0428
iteration 1
Model Alpha:  0.0426
iteration 2
Model Alpha:  0.0424
iteration 3
Model Alpha:  0.0422
iteration 4
Model Alpha:  0.042
iteration 5
Model Alpha:  0.041800000000000004
iteration 6
Model Alpha:  0.041600000000000005
iteration 7
Model Alpha:  0.041400000000000006
iteration 8
Model Alpha:  0.04120000000000001
iteration 9
Model Alpha:  0.04100000000000001
iteration 10
Model Alpha:  0.04080000000000001
iteration 11
Model Alpha:  0.04060000000000001
iteration 12
Model Alpha:  0.04040000000000001
iteration 13
Model Alpha:  0.040200000000000014
iteration 14
Model Alpha:  0.040000000000000015
iteration 15
Model Alpha:  0.039800000000000016
iteration 16
Model Alpha:  0.03960000000000002
iteration 17
Model Alpha:  0.03940000000000002
iteration 18
Model Alpha:  0.03920000000000002
iteration 19
Model Alpha:  0.03900000000000002
iteration 20
Model Alpha:  0.03880000000000002
iteration 21
Model Alpha:  0.03860000000000002
iteration 22
Model Alpha:  0.038400000000000024


In [16]:
#model= Doc2Vec.load("no_url.model_w4")
print("Model Loaded")

Model Loaded


In [42]:
model.most_similar('describe')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('explain', 0.6601998805999756),
 ('classify', 0.6462870240211487),
 ('define', 0.6090109944343567),
 ('teach', 0.601032018661499),
 ('understand', 0.5891581773757935),
 ('kill', 0.5884768962860107),
 ('consider', 0.584587574005127),
 ('introduce', 0.5781498551368713),
 ('express', 0.5779985189437866),
 ('confuse', 0.568020224571228)]

In [28]:
all_mbti.head()

,type,post,tokens,I,N,T,P,vectorized_comments
2,INFJ,enfp and intj moments urlweb sportscenter no...,"[enfp, and, intj, moments, urlweb, sportscente...",1,1,0,0,"[0.865215, 0.478191, 0.989163, -0.444672, -0.0..."
3,INFJ,what has been the most lifechanging experience...,"[what, has, been, the, most, lifechanging, exp...",1,1,0,0,"[-1.05232, 0.232657, -0.589661, 0.647971, 1.23..."
4,INFJ,urlweb urlweb on repeat for most of today,"[urlweb, urlweb, on, repeat, for, most, of, to...",1,1,0,0,"[-0.888199, 0.555337, -0.0629176, -0.0698107, ..."
5,INFJ,may the perc experience immerse you,"[may, the, perc, experience, immerse, you]",1,1,0,0,"[0.836576, 0.286395, -0.39377, -0.0151721, 0.0..."
6,INFJ,the last thing my infj friend posted on his fa...,"[the, last, thing, my, infj, friend, posted, o...",1,1,0,0,"[-0.397126, 0.741715, 0.521582, 1.40831, 1.104..."


In [3]:
#all_mbti.reset_index(inplace = True)
#all_mbti.drop('index',axis = 1,inplace =True)
all_mbti.reset_index(inplace = True)
all_mbti.head()
#all_mbti['Index'] = all_mbti['level_0']

NameError: name 'all_mbti' is not defined

In [161]:
para = 'P'

In [67]:
all_mbti.shape

(309301, 9)

In [68]:
adam = pd.concat([vc,all_mbti],axis = 1)

In [69]:
adam.shape

(309301, 109)

In [91]:
model.wv.similar_by_vector(ss)

[('cab', 0.3884442448616028),
 ('intjer', 0.354928195476532),
 ('rice', 0.34935396909713745),
 ('climbing', 0.3356604278087616),
 ('af', 0.3282245695590973),
 ('imperialism', 0.321328729391098),
 ('xd', 0.3191835284233093),
 ('jones', 0.3130473494529724),
 ('galaxy', 0.3112452030181885),
 ('phantom', 0.3050224781036377)]

In [2]:
model.wv.similar_by_word('soccer')

NameError: name 'model' is not defined